# Import necessary libraries

In [1]:
import json
import ast
import numpy as np
import pandas as pd
from itertools import groupby

from py_files.writer_director_to_one_hot import writer_director_to_one_hot
from py_files.add_merge_begin_end_year import merge_start_end_year
from py_files.load_box_office_data import load_and_aggregate_box_office
from py_files.add_remake_feature import create_remake_column
from py_files.add_langoriginaltitle_feature import add_language_of_original_title
from py_files.add_ENvsNonEN_feature import add_english_title_or_not
from py_files.add_movie_genre_feature import add_movie_genre
from py_files.df_processor_enrichment import df_processor_enrichment

from py_files.df_model_prep import df_model_prep
from py_files.d2v_embed import d2v_embed
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score
import math

# Loading the data

In [2]:
from py_files.load_original_data import load_original_data

df_original = load_original_data()

Found files: train-8.csv, train-2.csv, train-7.csv, train-5.csv, train-3.csv, train-4.csv, train-1.csv, train-6.csv


# Preprocessing of original columns

In [3]:
# copy the dataframe so we leave the original untouched
df_preprocessed = df_original.copy(deep=True)

# start the preprocessing
df_preprocessed = df_original.replace("\\N", np.nan)
df_preprocessed["primaryTitleFormatted"] = df_preprocessed["primaryTitle"].str.lower()\
                                                                          .str.normalize('NFKD')\
                                                                          .str.encode('ascii', errors='ignore')\
                                                                          .str.decode('utf-8')\
                                                                          .str.replace(" ", "_", regex=True)\
                                                                          .str.replace("\W", "", regex=True)

# merge endYear into beginYear when beginYear is not available --> rename Year
df_preprocessed = merge_start_end_year(df_preprocessed)

# set the datatypes of the dataframe correctly
df_preprocessed['Year'] = df_preprocessed['Year'].astype(int)
df_preprocessed['runtimeMinutes'] = df_preprocessed['runtimeMinutes'].astype(float)

df_preprocessed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7959 entries, 0 to 7958
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   tconst                 7959 non-null   object 
 1   primaryTitle           7959 non-null   object 
 2   originalTitle          3971 non-null   object 
 3   startYear              7173 non-null   object 
 4   endYear                786 non-null    object 
 5   runtimeMinutes         7946 non-null   float64
 6   numVotes               7169 non-null   float64
 7   label                  7959 non-null   bool   
 8   primaryTitleFormatted  7959 non-null   object 
 9   Year                   7959 non-null   int64  
dtypes: bool(1), float64(2), int64(1), object(6)
memory usage: 567.5+ KB


## Preprocessing of exogenous data

### Oscar data

In [4]:
oscars = pd.read_csv("additional_data/oscars.csv")

oscars["film"] = oscars["film"].str.lower()\
                               .str.normalize('NFKD')\
                               .str.encode('ascii', errors='ignore')\
                               .str.decode('utf-8')\
                               .str.replace(" ", "_", regex=True)\
                               .str.replace("\W", "", regex=True)

# Counting oscar nominations and wins per movie
oscar_noms = pd.merge(df_preprocessed, oscars, left_on = "primaryTitleFormatted", right_on = "film").groupby("tconst")["winner"].count()
oscar_wins = pd.merge(df_preprocessed, oscars, left_on = "primaryTitleFormatted", right_on = "film").groupby("tconst")["winner"].sum()

In [6]:
razzies = pd.read_csv("additional_data/Razzies.csv")

razzies["moviename"] = razzies["moviename"].str.lower()\
                               .str.normalize('NFKD')\
                               .str.encode('ascii', errors='ignore')\
                               .str.decode('utf-8')\
                               .str.replace(" ", "_", regex=True)\
                               .str.replace("\W", "", regex=True)

# Counting oscar nominations and wins per movie
razzie_noms = pd.merge(df_preprocessed, razzies, left_on = "primaryTitleFormatted", right_on = "moviename").groupby("tconst")["Wins"].count()
razzie_wins = pd.merge(df_preprocessed, razzies, left_on = "primaryTitleFormatted", right_on = "moviename").groupby("tconst")["Wins"].sum()

### Writer and Director data

In [7]:
# writers = writer_director_to_one_hot("writers")
# directors = writer_director_to_one_hot("directors")
# written_and_directed = writers.add(directors, fill_value=0).fillna(0).astype(int).loc[df_preprocessed["tconst"]]

### TMDB data

In [8]:
df_TMDB = pd.read_csv("additional_data/TMDB.csv")[["budget", "genres", "imdb_id", 
                                                   "original_language", "overview", 
                                                   "popularity", "production_companies", 
                                                   "tagline", "Keywords", "revenue"]]

In [9]:
def dict_to_list(dictionary):
    try:
        d = ast.literal_eval(dictionary)
    except ValueError:
        return []
    
    return [i["name"] for i in d]

In [10]:
df_TMDB["genres"] = df_TMDB["genres"].apply(lambda x: dict_to_list(x))
df_TMDB["Keywords"] = df_TMDB["Keywords"].apply(lambda x: dict_to_list(x))
df_TMDB["production_companies"] = df_TMDB["production_companies"].apply(lambda x: dict_to_list(x))
df_TMDB = df_TMDB.set_index("imdb_id")

### Metacritic data

In [11]:
df_meta = pd.read_csv("additional_data/Metacritic.csv").drop("Unnamed: 0", axis=1).set_index("movie")
df_meta["overview"] = df_meta["overview"].apply(lambda x: eval(x))
df_meta["overview"] = df_meta["overview"].apply(lambda x: x[0] if x else str(x))
df_meta.rename({'genres': 'genres_meta'}, axis=1, inplace=True)

In [12]:
# Combine overviews
overviews = pd.merge(df_TMDB["overview"], df_meta["overview"], left_index=True, right_index=True, how="outer")
overviews["overview"] = overviews["overview_x"].str.cat(overviews["overview_y"], na_rep="")
overviews = overviews.drop(["overview_x", "overview_y"], axis=1)
df_TMDB = df_TMDB.drop("overview", axis=1)

In [13]:
df_TMDB.rename({'genres': 'genres_tmdb'}, axis=1, inplace=True)

### Box Office data

In [14]:
df_box_office_mojo = load_and_aggregate_box_office()

# process the 'release group' (read movie title) in the same way as the formatted title
df_box_office_mojo["Release Group"] = df_box_office_mojo["Release Group"].str.lower()\
                                       .str.normalize('NFKD')\
                                       .str.encode('ascii', errors='ignore')\
                                       .str.decode('utf-8')\
                                       .str.replace(" ", "_", regex=True)\
                                       .str.replace("\W", "", regex=True)
df_box_office_mojo.drop(['%', '%.1'], axis=1, inplace=True)

Found files: box_office_mojo/2014.csv, box_office_mojo/1982.csv, box_office_mojo/1979.csv, box_office_mojo/1980.csv, box_office_mojo/2008.csv, box_office_mojo/1997.csv, box_office_mojo/2015.csv, box_office_mojo/1986.csv, box_office_mojo/2010.csv, box_office_mojo/1978.csv, box_office_mojo/1996.csv, box_office_mojo/2011.csv, box_office_mojo/1998.csv, box_office_mojo/2009.csv, box_office_mojo/2005.csv, box_office_mojo/2018.csv, box_office_mojo/1977.csv, box_office_mojo/1981.csv, box_office_mojo/1994.csv, box_office_mojo/2013.csv, box_office_mojo/2002.csv, box_office_mojo/2006.csv, box_office_mojo/1991.csv, box_office_mojo/1985.csv, box_office_mojo/2017.csv, box_office_mojo/1984.csv, box_office_mojo/1995.csv, box_office_mojo/2016.csv, box_office_mojo/2001.csv, box_office_mojo/2007.csv, box_office_mojo/1989.csv, box_office_mojo/1988.csv, box_office_mojo/2012.csv, box_office_mojo/2000.csv, box_office_mojo/1990.csv, box_office_mojo/2004.csv, box_office_mojo/2021.csv, box_office_mojo/2022.csv,

# Adding of exogenous columns

In [15]:
df_incl_exog = df_preprocessed.copy(deep=True)
df_incl_exog = df_incl_exog.rename({"tconst" : "id"}, axis = 1).set_index("id")
df_incl_exog.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7959 entries, tt0015224 to tt9900782
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   primaryTitle           7959 non-null   object 
 1   originalTitle          3971 non-null   object 
 2   startYear              7173 non-null   object 
 3   endYear                786 non-null    object 
 4   runtimeMinutes         7946 non-null   float64
 5   numVotes               7169 non-null   float64
 6   label                  7959 non-null   bool   
 7   primaryTitleFormatted  7959 non-null   object 
 8   Year                   7959 non-null   int64  
dtypes: bool(1), float64(2), int64(1), object(5)
memory usage: 567.4+ KB


## add oscar data

In [16]:
df_incl_exog["oscar_noms"] = oscar_noms
df_incl_exog["oscar_wins"] = oscar_wins

In [17]:
df_incl_exog["razzie_noms"] = razzie_noms
df_incl_exog["razzie_wins"] = razzie_wins

## add mojo box office

In [18]:
df_incl_exog = df_incl_exog.reset_index().merge(df_box_office_mojo, left_on=['primaryTitleFormatted', 'Year'], right_on=['Release Group', 'year'], how="left").set_index('id')
df_incl_exog.drop(['Release Group', 'year'], axis=1, inplace=True)

df_incl_exog.loc[df_incl_exog['Worldwide'] == '-', 'Worldwide'] = np.nan
df_incl_exog.loc[df_incl_exog['Domestic'] == '-', 'Domestic'] = np.nan
df_incl_exog.loc[df_incl_exog['Foreign'] == '-', 'Foreign'] = np.nan
df_incl_exog.loc[df_incl_exog['Worldwide'].notnull(), 'Worldwide'] = df_incl_exog.loc[df_incl_exog['Worldwide'].notnull(), 'Worldwide'].apply(lambda x: float(x.replace('$', '').replace(',', '')))
df_incl_exog.loc[df_incl_exog['Domestic'].notnull(), 'Domestic'] = df_incl_exog.loc[df_incl_exog['Domestic'].notnull(), 'Domestic'].apply(lambda x: float(x.replace('$', '').replace(',', '')))
df_incl_exog.loc[df_incl_exog['Foreign'].notnull(), 'Foreign'] = df_incl_exog.loc[df_incl_exog['Foreign'].notnull(), 'Foreign'].apply(lambda x: float(x.replace('$', '').replace(',', '')))

## add remake column

In [19]:
df_incl_exog = create_remake_column(df_incl_exog)

## add title language

In [20]:
# # add the language of the original title, currently commented for training data usage and not wait 15 min every time
# df_incl_exog = add_language_of_original_title(df_incl_exog)

df_added_lang = pd.read_csv('additional_data/df_added_lang.csv', index_col=0)
df_added_lang = df_added_lang.rename({"tconst" : "id"}, axis = 1).set_index("id")
df_incl_exog = df_incl_exog.join(df_added_lang['title_language'], how='left')

## add whether title is English or not

In [21]:
df_incl_exog = add_english_title_or_not(df_incl_exog)

## add movie genres

In [22]:
df_incl_exog = add_movie_genre(df_incl_exog)

## add writers and directors

In [23]:
# df_incl_exog = pd.concat([df_incl_exog.T, written_and_directed.T]).T

## add TMDB & Metacritic overviews

In [24]:
df_incl_exog = pd.merge(df_incl_exog, df_TMDB, how="left", left_index=True, right_index=True)
df_incl_exog = pd.merge(df_incl_exog, df_meta, how="left", left_index=True, right_index=True)
df_incl_exog = pd.merge(df_incl_exog, overviews, how="left", left_index=True, right_index=True)

In [25]:
df_incl_exog.columns

Index(['primaryTitle', 'originalTitle', 'startYear', 'endYear',
       'runtimeMinutes', 'numVotes', 'label', 'primaryTitleFormatted', 'Year',
       'oscar_noms', 'oscar_wins', 'razzie_noms', 'razzie_wins', 'Rank',
       'Worldwide', 'Domestic', 'Foreign', 'hasRemake', 'title_language',
       'isEN', 'year', 'titleFormatted', 'genres_movielens', 'budget',
       'genres_tmdb', 'original_language', 'popularity',
       'production_companies', 'tagline', 'Keywords', 'revenue', 'genres_meta',
       'language', 'overview_x', 'overview_y'],
      dtype='object')

## remove the last stuff we don't want

In [26]:
df_incl_exog['genres_tmdb']= df_incl_exog['genres_tmdb'].apply(lambda d: d if isinstance(d, list) else [])
df_incl_exog['genres_meta']= df_incl_exog['genres_meta'].apply(lambda d: d if isinstance(d, list) else [])
df_incl_exog['genres_movielens']= df_incl_exog['genres_movielens'].apply(lambda d: d if isinstance(d, list) else [])

### merge genre columns together

In [27]:
df_incl_exog['genres_combined'] = df_incl_exog['genres_meta'] + df_incl_exog['genres_tmdb'] + df_incl_exog['genres_movielens']
df_incl_exog['genres_combined'] = df_incl_exog['genres_combined'].apply(lambda x: list(set(x)))
df_incl_exog.drop(['genres_meta', 'genres_tmdb', 'genres_movielens'], axis=1, inplace=True, errors='ignore')


In [28]:
s = df_incl_exog['genres_combined'].explode()
df_incl_exog = df_incl_exog.join(pd.crosstab(s.index, s), how='left', lsuffix='x')

### coalesce revenue and box office values

In [29]:
df_incl_exog['revenue'] = df_incl_exog[['revenue', 'Worldwide']].max(axis=1)
df_incl_exog[df_incl_exog['revenue'] < 100000] = np.nan

In [31]:
# drop some unnecessary columns
df_incl_exog.drop(['startYear', 'endYear', 'primaryTitleFormatted', 'Domestic', 'Foreignx', 
                   'Keywords', 'genres_combined', 'overview_x', 'overview_y', 'tagline', 'Worldwide'], axis=1, inplace=True, errors='ignore')
df_incl_exog.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7959 entries, tt0015224 to tt9900782
Data columns (total 46 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   primaryTitle          7919 non-null   object 
 1   originalTitle         3950 non-null   object 
 2   runtimeMinutes        7906 non-null   float64
 3   numVotes              7135 non-null   float64
 4   label                 7919 non-null   object 
 5   Year                  7919 non-null   float64
 6   oscar_noms            892 non-null    float64
 7   oscar_wins            892 non-null    float64
 8   razzie_noms           84 non-null     float64
 9   razzie_wins           84 non-null     float64
 10  Rank                  1391 non-null   float64
 11  hasRemake             7919 non-null   object 
 12  title_language        7919 non-null   object 
 13  isEN                  7919 non-null   object 
 14  year                  3937 non-null   float64
 15  titleFormatte

### set columns with correct datatype

In [34]:
df_incl_exog['isEN'] = df_incl_exog['isEN'].astype('float')
df_incl_exog['title_language'] = df_incl_exog['title_language'].astype('string')

## save dataframe with features

In [ ]:
df_incl_exog.to_csv('df_with_features.csv')

# Preparing data for classifier

Convert non-numeric columns to numeric.
We use Doc2Vec to embed each string column into n-by-128 array 

In [ ]:
# from py_files.df_processor_enrichment import df_processor_enrichment
# train_df = pd.read_csv('df_with_features.csv', index_col=0)
train_df = df_processor_enrichment('train')
train_df.head()

In [ ]:
train_df["startYear"] = train_df["startYear"].astype(float)
train_df["endYear"] = train_df["endYear"].astype(float)

In [ ]:
# from py_files.df_model_prep import df_model_prep

train_df_prepped = df_model_prep(train_df,'train')
train_df_prepped.head()

# Evaluating classifier

In [ ]:
model_lgbm = lgb.LGBMClassifier(objective='binary',
                                learning_rate=0.01,
                                num_iterations=1000,
                                feature_fraction=0.8,
                                verbosity=1,
                                random_state=17)
model_lgbm.fit(train_df_prepped.loc[:, train_df_prepped.columns != 'label'],
              train_df_prepped['label'],
              eval_metric='logloss')

# Predicting

## Add and process train and valid data

In [ ]:
valid_df = df_processor_enrichment('validation_hidden.csv')
valid_df.head()

In [ ]:
valid_df["startYear"] = train_df["startYear"].astype(float)
valid_df["endYear"] = train_df["endYear"].astype(float)

In [ ]:
valid_df_prepped = df_model_prep(valid_df, 'valid')
valid_df_prepped.head()

In [ ]:
valid_df_prepped.info()

In [ ]:
test_df = df_processor_enrichment('test_hidden.csv')
test_df.head()

In [ ]:
test_df.info()

In [ ]:
test_df_prepped = df_model_prep(test_df, 'test')
test_df_prepped.head()

In [ ]:
test_df_prepped.info()

In [ ]:
val_preds_lgbm = model_lgbm.predict(valid_df_prepped)
test_preds_lgbm = model_lgbm.predict(test_df_prepped)

In [ ]:
with open('val_preds_lgbm.txt', 'w+') as f:
    for val in val_preds_lgbm:
        f.write(f"{str(val)}\n")

In [ ]:
with open('test_preds_lgbm.txt', 'w+') as f:
    for val in test_preds_lgbm:
        f.write(f"{str(val)}\n")